In [1]:
import nltk
nltk.download(u'wordnet')

[nltk_data] Error loading stopword: Package u'stopword' not found in
[nltk_data]     index
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
from nltk.stem.porter import *
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import wordnet
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from datetime import datetime
import re
from collections import defaultdict
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from datetime import datetime
from sklearn.cross_validation import StratifiedShuffleSplit
np.random.seed(1337) 

class DFtransformer(object):
    def __init__(self,min_df=5):
        self.min_df = min_df
        self.Vocab = defaultdict(int)
        self.Vocab_ = {}
    def fit(self,sentence_lst):
        for sentence in sentence_lst:
            for word in sentence.split():
                self.Vocab[word] += 1
        print ("origenal vocab:",len(self.Vocab))
        self.Vocab = {k:v for k,v in self.Vocab.items() if self.Vocab[k]>=self.min_df}   
        i = 1
        for k in self.Vocab:
            self.Vocab_[k] = i
            i += 1
        print ("transformed vocab:",len(self.Vocab))
        
    def tranform(self,sentence_lst):
        ret_s_lst = []
        for s in sentence_lst:
            ret_s = np.array([self.Vocab_[w] for w in s.split() if w in self.Vocab_])
            #print ret_s
            ret_s_lst.append(ret_s)
        return np.array(ret_s_lst)

toker = TreebankWordTokenizer()
lemmer = wordnet.WordNetLemmatizer()
stemmer = PorterStemmer()

stop_w = set(stopwords.words('english'))
# w2v_model =  '../../../../media/lhc/B2DA42D1DA429191/word2vec_data/GoogleNews-vectors-negative300.bin'
# embedder = Word2Vec.load_word2vec_format(w2v_model,binary=True)

def str_lem(s):
    s = s.lower()
    s = (" ").join([z for z in re.findall('\w{2,}',s) if z not in stop_w])
    s = (" ").join([lemmer.lemmatize(z) for z in s.split(" ")])
    return s
def str_stem(s):
    s = s.lower()
    s = (" ").join([z for z in re.findall('\w{2,}',s) if z not in stop_w])
    s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
    return s
def apply_text_preprocessor(columns,str_process):
    rets = []
    for i,row in enumerate(columns,start=1):
#         if i%10000==0:
#             print i,
        rets.append(str_process(row))
    return rets

Using Theano backend.
Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN not available)


In [2]:
fin = './data/news_o1.csv'
df_all = pd.read_csv(fin,encoding='utf8')
#df_all = df_all[df_all.target.notnull()]
df_all['title_stem'] = apply_text_preprocessor(df_all['title'],str_stem)
df_all['title_lem'] = apply_text_preprocessor(df_all['title'],str_lem)
df_all.head()
#test
# df_ = df_all[df_all['ticker'] == 'AAPL']
# df_.index = df_['date'].map(pd.Timestamp)
# df_['2016-04-25':'2016-04-30']

,Date,d1,d2,d3,date,title,ticker,title_stem,title_lem
0,2014-05-15,0.009910,-0.001090,-0.012007,2014-05-15,"US STOCKS-Dow, S&P 500 end at record highs aga...",A,us stock dow 500 end record high googl jump,u stock dow 500 end record high google jump
1,2014-07-03,-0.005816,-0.016690,-0.004375,2014-07-03,"Hollande, Merkel urge Putin to broker Ukraine ...",A,holland merkel urg putin broker ukrain ceasefir,hollande merkel urge putin broker ukraine ceas...
2,2014-11-17,-0.010669,0.000000,0.011274,2014-11-17,Agilent beats 4Q profit forecasts,A,agil beat 4q profit forecast,agilent beat 4q profit forecast
3,2015-02-17,0.025173,-0.009389,0.019441,2015-02-17,Agilent matches Street 1Q forecasts,A,agil match street 1q forecast,agilent match street 1q forecast
4,2015-03-02,-0.010305,-0.006152,0.005238,2015-03-02,"Mallinckrodt PLC (MNK), Pfizer Inc. (PFE): Hea...",A,mallinckrodt plc mnk pfizer inc pfe healthcor ...,mallinckrodt plc mnk pfizer inc pfe healthcor ...


In [3]:
target = 'd1'

df = df_all[df_all[target].notnull()]
dftransformer =  DFtransformer(min_df=5)
dftransformer.fit(df['title_lem'])
seq_lst = dftransformer.tranform(df['title_lem'])
maxlen = 20
X = sequence.pad_sequences(seq_lst,maxlen=maxlen)
y = (df[target].values>0).astype(np.int64)

origenal vocab: 40593
transformed vocab: 15942


In [4]:
max_features = 15942 + 1
batch_size = 32

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (242941, 20)
X_test shape: (60736, 20)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0 10321
 13493 12780 13735  5949 13172 15240 14262  2430]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 20, 128)       2040704     embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 128)           131584      embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             129         lstm_1[0][0]                     
___________________________________________________________________________

KeyboardInterrupt: 

In [6]:
max_features = 15942 + 1
batch_size = 1000

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (242941, 20)
X_test shape: (60736, 20)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0 10321
 13493 12780 13735  5949 13172 15240 14262  2430]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_3 (Embedding)          (None, 20, 128)       2040704     embedding_input_3[0][0]          
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (None, 128)           131584      embedding_3[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 1)             129         lstm_3[0][0]                     
___________________________________________________________________________

KeyboardInterrupt: 

In [10]:
max_features = 15942 + 1
batch_size = 3200

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2, consume_less='gpu'))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (242941, 20)
X_test shape: (60736, 20)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0 10321
 13493 12780 13735  5949 13172 15240 14262  2430]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, 20, 128)       2040704     embedding_input_6[0][0]          
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (None, 128)           131584      embedding_6[0][0]                
____________________________________________________________________________________________________
dense_6 (Dense)                  (None, 1)             129         lstm_6[0][0]                     
___________________________________________________________________________

In [11]:
max_features = 15942 + 1
batch_size = 3200

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2, consume_less='gpu'))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (242941, 20)
X_test shape: (60736, 20)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0 10321
 13493 12780 13735  5949 13172 15240 14262  2430]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_7 (Embedding)          (None, 20, 256)       4081408     embedding_input_7[0][0]          
____________________________________________________________________________________________________
lstm_7 (LSTM)                    (None, 128)           197120      embedding_7[0][0]                
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 1)             129         lstm_7[0][0]                     
___________________________________________________________________________

In [12]:
max_features = 15942 + 1
batch_size = 3200

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen, dropout=0.2))
model.add(LSTM(256, dropout_W=0.2, dropout_U=0.2, consume_less='gpu'))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (242941, 20)
X_test shape: (60736, 20)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0 10321
 13493 12780 13735  5949 13172 15240 14262  2430]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_8 (Embedding)          (None, 20, 256)       4081408     embedding_input_8[0][0]          
____________________________________________________________________________________________________
lstm_8 (LSTM)                    (None, 256)           525312      embedding_8[0][0]                
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 1)             257         lstm_8[0][0]                     
___________________________________________________________________________

In [13]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)

Train on 242941 samples, validate on 60736 samples
Epoch 1/15
242941/242941 [==============================] - 37s - loss: 0.5686 - acc: 0.6677 - val_loss: 0.7542 - val_acc: 0.5676
Epoch 2/15
242941/242941 [==============================] - 37s - loss: 0.5624 - acc: 0.6736 - val_loss: 0.7689 - val_acc: 0.5696
Epoch 3/15
242941/242941 [==============================] - 37s - loss: 0.5566 - acc: 0.6778 - val_loss: 0.7698 - val_acc: 0.5717
Epoch 4/15
242941/242941 [==============================] - 37s - loss: 0.5521 - acc: 0.6821 - val_loss: 0.7742 - val_acc: 0.5714
Epoch 5/15
242941/242941 [==============================] - 37s - loss: 0.5470 - acc: 0.6862 - val_loss: 0.7780 - val_acc: 0.5726
Epoch 6/15
242941/242941 [==============================] - 37s - loss: 0.5405 - acc: 0.6920 - val_loss: 0.7881 - val_acc: 0.5748
Epoch 7/15
242941/242941 [==============================] - 37s - loss: 0.5358 - acc: 0.6958 - val_loss: 0.7922 - val_acc: 0.5749
Epoch 8/15
242941/242941 [=============

In [14]:
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test),verbose=1)

Train on 242941 samples, validate on 60736 samples
Epoch 1/15
242941/242941 [==============================] - 37s - loss: 0.4991 - acc: 0.7264 - val_loss: 0.8466 - val_acc: 0.5831
Epoch 2/15
242941/242941 [==============================] - 37s - loss: 0.4930 - acc: 0.7310 - val_loss: 0.8631 - val_acc: 0.5836
Epoch 3/15
242941/242941 [==============================] - 37s - loss: 0.4909 - acc: 0.7332 - val_loss: 0.8535 - val_acc: 0.5851
Epoch 4/15
242941/242941 [==============================] - 37s - loss: 0.4869 - acc: 0.7349 - val_loss: 0.8526 - val_acc: 0.5863
Epoch 5/15
242941/242941 [==============================] - 37s - loss: 0.4830 - acc: 0.7375 - val_loss: 0.8591 - val_acc: 0.5854
Epoch 6/15
242941/242941 [==============================] - 37s - loss: 0.4802 - acc: 0.7395 - val_loss: 0.8667 - val_acc: 0.5856
Epoch 7/15
242941/242941 [==============================] - 37s - loss: 0.4779 - acc: 0.7425 - val_loss: 0.8741 - val_acc: 0.5851
Epoch 8/15
242941/242941 [=============

In [15]:
fin = './data/news_o2.csv'
df_all = pd.read_csv(fin,encoding='utf8')
#df_all = df_all[df_all.target.notnull()]
df_all['title_stem'] = apply_text_preprocessor(df_all['title'],str_stem)
df_all['title_lem'] = apply_text_preprocessor(df_all['title'],str_lem)
df_all.head()

,Date,d1,d2,d3,date,title,ticker,title_stem,title_lem
0,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Alcoa (AA) to Shutter Pocos de Caldas Smelter ...,AA,alcoa aa shutter poco de calda smelter brazil ...,alcoa aa shutter pocos de caldas smelter brazi...
1,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Q2 Earnings Season in the Spotlight with Alcoa...,AA,q2 earn season spotlight alcoa report earn pre...,q2 earnings season spotlight alcoa report earn...
2,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,Earnings will be a nail biter: Jack Ablin,AA,earn nail biter jack ablin,earnings nail biter jack ablin
3,2015-07-02,-0.006306,0.00272,-0.050633,2015-07-02,3 Beaten-Down Stocks That May Pay Off for Cont...,AA,beaten stock may pay contrarian investor,beaten stock may pay contrarian investor
4,NaN,NaN,NaN,NaN,2015-07-03,UPDATE 1-Japan Q3 aluminium premiums mostly se...,AA,updat japan q3 aluminium premium mostli set ye...,update japan q3 aluminium premium mostly set y...


In [16]:
target = 'd1'

df = df_all[df_all[target].notnull()]
dftransformer =  DFtransformer(min_df=5)
dftransformer.fit(df['title_lem'])
seq_lst = dftransformer.tranform(df['title_lem'])
maxlen = 20
X = sequence.pad_sequences(seq_lst,maxlen=maxlen)
y = (df[target].values>0).astype(np.int64)

origenal vocab: 24651
transformed vocab: 9290


In [ ]:
max_features = 9290 + 1
batch_size = 320

for tr,va in StratifiedShuffleSplit(y,train_size=0.8,test_size=0.2,random_state=1024,n_iter=1):
    X_train = X[tr]
    y_train = y[tr]
    X_test = X[va]
    y_test = y[va]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('X_train[0]:',X_train[0])
print('y_train[0]:',y_train[0])

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 256, input_length=maxlen, dropout=0.2))
model.add(LSTM(256, dropout_W=0.2, dropout_U=0.2, consume_less='gpu'))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# for n in range(15):
#     model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1,
#               validation_data=(X_test, y_test),verbose=1)
#     score, acc = model.evaluate(X_train, y_train,verbose=1,
#                                 batch_size=batch_size)
#     print()
#     print('iter:',n+1)
#     print('Train score:', score)
#     print('Train accuracy:', acc)
#     score, acc = model.evaluate(X_test, y_test,verbose=1,
#                                 batch_size=batch_size)
#     print('Test score:', score)
#     print('Test accuracy:', acc)
    
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=30,
          validation_data=(X_test, y_test),verbose=1)
score, acc = model.evaluate(X_train, y_train,verbose=1,
                            batch_size=batch_size)

X_train shape: (86389, 20)
X_test shape: (21598, 20)
X_train[0]: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
  424 2569 5021 1320 3521]
y_train[0]: 0
Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_9 (Embedding)          (None, 20, 256)       2378496     embedding_input_9[0][0]          
____________________________________________________________________________________________________
lstm_9 (LSTM)                    (None, 256)           525312      embedding_9[0][0]                
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 1)             257         lstm_9[0][0]                     
________________________________________________________________________________________________